# 🦜🔗 LangChain核心源代码解读

# 课程开始

## ❤️ 这节课会带给你

1. 🌹一起阅读 Langchain 核心组件的源码：Runnable、LLM、Agent、Langgraph
2. 🌹从解读源码的角度了解 Langchain 整体设计思路，以便更好地阅读官方文档
3. 🌹掌握 Langchain 文档中未曾提及、翻看源码才知晓的实用技巧
4. ✍️ 动手实现一个自定义大模型框架：会说话的邻居老大爷
5. ✍️ 动手集成自己的大模型到 LangChain：智谱AI
6. ✍️ 动手实现基于 LCEL 的智能体：再现手撕AutoGPT
7. ✍️ 动手实现基于 Langgraph 的智能体：再现手撕AutoGPT

<div class="alert alert-warning">
<b>注意：</b><br>
<ul>
    <li>大模型认知：请参考AGI课堂正课相关章节</li>
    <li>Langchain 基础：请参考AGI课堂正课相关章节</li>
    <li>看懂源码是为了写好代码</li>
</ul> 
</div>

## 如何解读 Langchain 的源代码结构

### 1、你有哪些资源可以利用？

![](./langchain_ai.png)

（1）学习资源：
- [langchain源代码](https://github.com/langchain-ai/)：All you need r here !!!
- [langchain官方文档](https://python.langchain.com/docs)：与源代码相互印证
- [langgraph example](https://github.com/langchain-ai/langgraph/tree/main/examples)：Jupyter Notes

（2）良师益友：
- langchain [聊天](https://chat.langchain.com/?llm=anthropic_claude_2_1)：免费的大模型+RAG（也可以学习其源码）
- Github Copilot：程序员无法离开的工具，就像现在的人开车无法离开地图导航

### 2、🌹 阅读源码：LangChain 源代码概览

[langchain源代码结构](https://github.com/langchain-ai/langchain/tree/master/libs)

| 源码位置 | 功能描述 |
| :--- | :--- |
| langchain/libs/langchain | 模块入口，会导入core、community等其他模块 |
| langchain/libs/core | 核心组件和关键的基类实现 |
| langchain/libs/partners | 合作伙伴（官方合作）组件 |
| langchain/libs/community | 社区（非官方）组件 |
| langchain/libs/experimental | 试验性功能（前沿探索组件，不对版本稳定做承诺） |


### 3、LangChain核心框架的三轮迭代

- Runnable + Chain 时代
- Runnable + LCEL 时代
- Runnable + LCEL + Langgraph 时代

# （一）从零开始集成大模型到 Langchain 实例

In [1]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 大模型的一般用法回顾

In [34]:
# LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [35]:
# invoke
text = "请帮我想一想，生产一款「基于AI自动写字的铅笔」的公司有什么好名字?"
response = llm.invoke(text)
print(response.content)

1. 智能字铅
2. AI文笔
3. 自动书写
4. 涂字智能
5. 字迹神笔
6. AI字画
7. 自动书法
8. 智能写字
9. 自动笔迹
10. AI字迹通


In [36]:
# stream
for chunk in llm.stream(text):
    print(chunk.content, end="|", flush=True)

|1|.| 智|能|笔|迹|
|2|.| 创|意|笔|锋|
|3|.| AI|写|字|宝|
|4|.| 涂|鸦|助|手|
|5|.| 文|字|小|帮|手|
|6|.| 心|灵|笔|耕|
|7|.| 智|能|书|写|
|8|.| 创|造|笔|迹|
|9|.| 智|能|铅|笔|坊|
|10|.| 文|字|魔|法|使||

In [37]:
# LCEL LLM+Prompt+Outputparser
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate.from_template("请帮我想一想，生产一款{product}的公司有什么好名字?")
chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"product": "基于AI自动写字的铅笔"}):
    print(chunk, end="|", flush=True)

|1|.| 智|能|笔|记|
|2|.| AI|笔|迹|
|3|.| 心|灵|笔|
|4|.| 智|慧|铅|笔|
|5|.| 创|意|笔|迹|
|6|.| 自|动|笔|画|
|7|.| 智|能|写|字|
|8|.| AI|手|写|笔|
|9|.| 智|能|文字|
|10|.| 创|意|笔|锋||

<div class="alert alert-warning">
    <b>⚠️ 思考</b><br>
    langchain 支持的8个方法都在什么场景下使用？
</div>


## ❤️ 实例1 实现「楼下邻居老大爷」AI大模型

### 🦜 需求分析：参考电影片段，把「非常有智慧的楼下邻居老大爷」变成 AI大模型

1. 使用大模型：模拟一个大模型
2. 生成能力：提及马冬梅时生成打岔闲聊（马什么梅？马冬什么？什么冬梅？），其余生成“哦...“

![马冬梅](./madongmei.gif)

### 🌹 看源码：

#### （1）BaseLanguageModel

来自：[https://github.com/langchain-ai/langchain](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/base.py#L74-L81)

```python
class BaseLanguageModel(
    RunnableSerializable[LanguageModelInput, LanguageModelOutputVar], ABC
):
    """Abstract base class for interfacing with language models.

    All language model wrappers inherit from BaseLanguageModel.
    """
```

#### （2）BaseChatModel

来自：[https://github.com/langchain-ai/langchain](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/chat_models.py#L100)

**核心逻辑：**

```python
class BaseChatModel(BaseLanguageModel[BaseMessage], ABC):
    """Base class for Chat models."""

    ...

    def invoke(...) -> BaseMessage:
        # ...
        self.generate_prompt(...)
        # generate_prompt >> generate >> _generate

    def ainvoke(...) -> BaseMessage:
        # ...
        self.agenerate_prompt(...)
        # agenerate_prompt >> agenerate >> _agenerate >> _generate
    
    def stream(...) -> Iterator[BaseMessageChunk]:
        # ...
        if type(self)._stream == BaseChatModel._stream:
            # model doesn't implement streaming, so use default implementation
            yield cast(
                BaseMessageChunk, self.invoke(input, config=config, stop=stop, **kwargs)
            )
        # ...

    async def astream(...) -> AsyncIterator[BaseMessageChunk]:
        # 在#19332合并中，_astream方法实现已经被简化
        # https://github.com/langchain-ai/langchain/pull/19332/commits/afbe6ac659e41ab5f4a6f4dcaa33511e9e59e4d5
        if (
            type(self)._astream is BaseChatModel._astream
            and type(self)._stream is BaseChatModel._stream
        ):
            # No async or sync stream is implemented, so fall back to ainvoke
            yield cast(
                BaseMessageChunk,
                await self.ainvoke(input, config=config, stop=stop, **kwargs),
            )
        # ...

    # bacth, abatch, astream_log, astream_events 
    # ...
```

**必须实现的部分：**

```python
    ## ******** invoke / ainvoke / batch / abatch **********
    @abstractmethod
    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Top Level call"""

    ## ******** stream / astream / astream_log / astream_events **********
    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        raise NotImplementedError()        
```

### ✍️ 基本实现：invoke

In [4]:
import time
import re
from typing import Any, Dict, Iterator, List, Optional, Union
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, HumanMessageChunk, AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

In [152]:
class ChatWithOlderAI(BaseChatModel):
    """模拟跟马冬梅楼下邻居老大爷的对话"""

    responses: List[BaseMessage] = [HumanMessage(m) for m in [
        "马什么梅？", "什么冬梅？？", "马东什么？？？",
    ]]
    
    sleep: Optional[float] = 0.1
    i: int = 0

    # 必须实现
    @property
    def _llm_type(self) -> str:
        return "chat-with-neighber-older"

    # 必须实现
    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        generations = [ChatGeneration(message=res) for res in self._ask_remote(messages)]
        return ChatResult(generations=generations)

    # 访问远程大模型
    def _ask_remote(self, messages: List[BaseMessage]) -> List[BaseMessage]:
        if(re.search("马冬梅", messages[0].content)):
            response = self.responses[self.i]
            if self.i < len(self.responses) - 1:
                self.i += 1
            else:
                self.i = 0
        else:
            response = AIMessage("哦...")
        return [response]

In [41]:
questions = [[HumanMessage(m)] for m in [
    "大爷，楼上322住的是马冬梅家吗？",
    "马冬梅啊",
    "马冬梅！",
    "我是说马冬梅！",
    "您歇着吧..."
]]

llm = ChatWithOlderAI()

for question in questions:
    print(f"\n\n夏洛：{question[0].content}")
    print("大爷：", end="")
    # print(llm.invoke(question).content, end="")
    for chunk in llm.stream(question):
        print(chunk.content, end="|")



夏洛：大爷，楼上322住的是马冬梅家吗？
大爷：马什么梅？|

夏洛：马冬梅啊
大爷：什么冬梅？？|

夏洛：马冬梅！
大爷：马东什么？？？|

夏洛：我是说马冬梅！
大爷：马什么梅？|

夏洛：您歇着吧...
大爷：哦...|

### ✍️ 支持流式输出：stream

In [42]:
class StreamChatWithOlderAI(ChatWithOlderAI):
    """模拟跟大爷的对话，支持流"""

    def _stream(
        self,
        messages: List[BaseMessage],
        *args,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        response = self._ask_remote(messages)
        for chunk in response[0].content:
            if self.sleep is not None:
                time.sleep(self.sleep)
            yield ChatGenerationChunk(message=AIMessageChunk(content=chunk))

In [43]:
llm_stream = StreamChatWithOlderAI()

for question in questions:
    print(f"\n\n夏洛：{question[0].content}")
    print("大爷：", end="")
    for chunk in llm_stream.stream(question):
        print(chunk.content, end="|")



夏洛：大爷，楼上322住的是马冬梅家吗？
大爷：马|什|么|梅|？|

夏洛：马冬梅啊
大爷：什|么|冬|梅|？|？|

夏洛：马冬梅！
大爷：马|东|什|么|？|？|？|

夏洛：我是说马冬梅！
大爷：马|什|么|梅|？|

夏洛：您歇着吧...
大爷：哦|.|.|.|

In [44]:
# Prompt
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

questions_text = [
    "楼上322住的是马冬梅家吗？",
    "马冬梅啊",
    "马冬梅！",
    "我是说马冬梅！",
    "大爷您歇着吧..."
]

llm_stream = StreamChatWithOlderAI()
prompt = PromptTemplate.from_template("大爷，{question}")
chain = prompt | llm_stream | StrOutputParser()

for question in questions_text:
    print(f"\n\n夏洛：{question}")
    print("大爷：", end="")
    for chunk in chain.stream({"question": question}):
        print(chunk, end="|")



夏洛：楼上322住的是马冬梅家吗？
大爷：马|什|么|梅|？|

夏洛：马冬梅啊
大爷：什|么|冬|梅|？|？|

夏洛：马冬梅！
大爷：马|东|什|么|？|？|？|

夏洛：我是说马冬梅！
大爷：马|什|么|梅|？|

夏洛：大爷您歇着吧...
大爷：哦|.|.|.|

### 🌹 审视这个模拟大模型

In [66]:
chain.get_graph().print_ascii()
# llm_stream.input_schema.schema()
# chain.input_schema.schema()
# prompt.output_schema.schema()
# prompt.invoke({"question":"你好"})

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
+-----------------------+  
| StreamChatWithOlderAI |  
+-----------------------+  
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## ❤️ 实例2 集成智谱大模型到 Langchain

### 🦜 需求分析：结合智谱官方文档，编写langchain的ChatZhipuAI

[访问智谱AI官方的Python接口文档](https://maas.aminer.cn/dev/api#sdk)

#### （1）可用接口
- 直接调用：invoke
- 异步调用（先调用，再查询结果）：适合实现 ainvoke / batch / abatch
- 流式调用（SSE，Server-Send Events)：适合实现 stream / astream / stream_log / stream_events

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    在哪些场景下可以充分利用官方API提供的异步能力？
</div>

#### （2）支持能力
- 支持 工具回调：形式上支持多工具返回
- 支持 识别图片
- 支持 生成图片

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    能否实现一个自定义大模型，支持多模态能力？
</div>

#### （3）速率限制

[智谱AI官方文档中对速率限制的说明](https://maas.aminer.cn/dev/howuse/rate-limits/why?tab=5)：

> 当前我们基于用户的月度 API 调用消耗金额情况将速率控制分为6种等级。
>
> 消耗金额选取逻辑：我们会选取用户当前月份1号～t-1日的调用 API 推理消耗总金额和用户上个月的 API 调用消耗总金额做比较，取更高金额作为用户当前的 API 消耗金额。
>
> 特别的，若您从未曾付费充值/购买过资源包，则会归为免费级别。

**整理GLM4模型使用限制如下：**
|用户等级|使用量|GLM4并发限制|
|:---|:---|:---|
|免费|api调用消耗0元-50元/每月（不含）|5|
|使用量1|api调用消耗50元-500元/每月（不含）|10|
|使用量2|api调用消耗500元-5000元/每月（不含）|20|
|使用量3|api调用消耗5000元-10000元/每月（不含）|30|
|使用量4|api调用消耗10000元-30000元/每月（不含）|100|
|使用量5|api调用消耗30000元以上/每月|200|

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    能否实现一个自定义大模型，通过多个低用量账户管理机制来扩大调用速率？
</div>

### ✍️ 测试官方例子

看官方例子：[https://github.com/MetaGLM/zhipuai-sdk-python-v4](https://github.com/MetaGLM/zhipuai-sdk-python-v4)

In [75]:
from langchain_zhipu import ZhipuAI
# from zhipuai import ZhipuAI

In [93]:
## 看看官方的例子是否能正确运行
client = ZhipuAI()
response = client.chat.completions.create(
    model="glm-4",
    messages=[
        {"role": "user", "content": "你叫什么名字"},
    ],
)
print(response.choices[0].message)

content='你好，我叫智谱清言，是基于智谱AI公司于2023年训练的ChatGLM开发的。很高兴见到你，欢迎问我任何问题。' role='assistant' tool_calls=None


### ✍️ 包装为一个 函数调用

In [101]:
def ask_zhipu(question) -> str:
    client = ZhipuAI()

    messages = [
        {"role": "system", "content": "你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。"},
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "hello"},
        {"role": "user", "content": question},
    ]

    response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
    )
    return(response.choices[0].message.content)

ask_zhipu("你叫什么名字？")

'What is your name?'

### ✍️ 支持与 Prompt 协作

```python
# 能否实现如下场景？
chain = prompt | llm
chain.invoke({"question": "你叫什么名字？"})
```

<div class="alert alert-warning">
    <b>⚠️ 思考</b><br>
    下面代码中， 为什么 from_messages 支持 system、human、ai 这些名字？还有其他名字吗？文档在哪里？
</div>

**🌞 答案：**
- [查看 _create_message_from_message_type() 源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/messages/utils.py#L130-L168)

In [276]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。"),
    ("human", "你好"),
    ("ai", "hello"),
    ("human", "{question}"),
])
prompt.invoke({"question":"你叫什名字？"})

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。'), HumanMessage(content='你好'), AIMessage(content='hello'), HumanMessage(content='你叫什名字？')])

In [277]:
prompt.invoke({"question":"你叫什名字？"}).to_messages()

[SystemMessage(content='你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。'),
 HumanMessage(content='你好'),
 AIMessage(content='hello'),
 HumanMessage(content='你叫什名字？')]

In [278]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage

def convert_message_to_dict(message: BaseMessage) -> dict:
    """从langchain消息格式转换为大模型字典格式"""
    message_dict: Dict[str, Any]
    if isinstance(message, HumanMessage):
        message_dict = {"role": "user", "content": message.content}
    elif isinstance(message, AIMessage):
        message_dict = {"role": "assistant", "content": message.content}
        if "tool_calls" in message.additional_kwargs:
            message_dict["tool_calls"] = message.additional_kwargs["tool_calls"]
    elif isinstance(message, SystemMessage):
        message_dict = {"role": "system", "content": message.content}
    elif isinstance(message, ToolMessage):
        message_dict = {
            "role": "tool",
            "content": message.content,
            "tool_call_id": message.tool_call_id,
        }
    else:
        raise TypeError(f"Unknown type from langchain to LLM: {type(message).__name__} {message}")
    return message_dict

In [279]:
[convert_message_to_dict(m) for m in prompt.invoke({"question":"你叫什名字？"}).to_messages()]

[{'role': 'system',
  'content': '你是一个翻译机器人，我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。'},
 {'role': 'user', 'content': '你好'},
 {'role': 'assistant', 'content': 'hello'},
 {'role': 'user', 'content': '你叫什名字？'}]

### ✍️ 包装为一个 RunnableLambda

In [280]:
from langchain_core.runnables import chain
from typing import List
from langchain_core.prompt_values import ChatPromptValue 

@chain
def ask_zhipu(promptValue: ChatPromptValue) -> str:
    client = ZhipuAI()
    response = client.chat.completions.create(
        model="glm-4",
        messages=[convert_message_to_dict(m) for m in promptValue.to_messages()],
    )
    return(response.choices[0].message.content)

In [281]:
chain = prompt | ask_zhipu
chain.invoke({"question": "你叫什名字？"})

'What is your name?'

In [282]:
# 看看当前链的结构
chain.get_graph().print_ascii()

    +-------------+    
    | PromptInput |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
+-------------------+  
| Lambda(ask_zhipu) |  
+-------------------+  
           *           
           *           
           *           
 +------------------+  
 | ask_zhipu_output |  
 +------------------+  


### ✍️ 基于BaseChatModel实现

In [283]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from typing import Any, Dict, Iterator, List, Optional, cast, Mapping
from langchain_core.messages import (
    HumanMessage,
    AIMessage,
    BaseMessage, 
    SystemMessage,
    ToolMessage,
)
from langchain_core.outputs import (
    ChatGeneration,
    ChatResult
)

In [284]:
def convert_dict_to_message(_dict: Mapping[str, Any]) -> BaseMessage:
    """从大模型字典格式转换为langchain消息格式"""
    role = _dict.get("role")
    if role == "user":
        return HumanMessage(content=_dict.get("content", ""))
    elif role == "assistant":
        content = _dict.get("content", "") or ""
        additional_kwargs: Dict = {}
        if tool_calls := _dict.get("tool_calls"):
            additional_kwargs["tool_calls"] = tool_calls
        return AIMessage(content=content, additional_kwargs=additional_kwargs)
    elif role == "system":
        return SystemMessage(content=_dict.get("content", ""))
    elif role == "tool":
        additional_kwargs = {}
        if "name" in _dict:
            additional_kwargs["name"] = _dict["name"]
        return ToolMessage(
            content=_dict.get("content", ""),
            tool_call_id=_dict.get("tool_call_id"),
            additional_kwargs=additional_kwargs,
            id=id_,
        )
    else:
        raise TypeError(f"Unknow type from LLM to langchain: {_dict}")

In [285]:
class MiniZhipuAI(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None

    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self.client = ZhipuAI()

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _ask_remote(self, messages, streaming=False, **kwargs):
        # 从langchain消息格式，转换到智谱AI输入的格式
        dict_zhipu = [convert_message_to_dict(m) for m in messages]
        
        response = self.client.chat.completions.create(
            model="glm-4",
            messages=dict_zhipu,
            stream=streaming,
            **kwargs
        )

        # 从智谱AI输出的格式，转换到langchain的消息格式
        if not isinstance(response, dict):
            response = response.dict()
        return [convert_dict_to_message(c["message"]) for c in response["choices"]]

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """实现 ZhiputAI 的同步调用"""

        # 问智谱AI，并得到回复
        responses = self._ask_remote(messages, streaming=False, **kwargs)

        return ChatResult(
            generations=[ChatGeneration(message=m) for m in responses]
        )

In [286]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个中英互译机器人，只负责翻译，不要试图对问题做解答。我说中文你就直接翻译成英文，我说英文你就直接翻译为中文。不要输出其他，不要啰嗦。"),
    ("human", "你好"),
    ("ai", "hello"),
    ("human", "{question}"),
])
llm_zhipu = MiniZhipuAI()
chain = prompt | llm_zhipu

chain.invoke({"question": "The competition between China and the United States in the AI field is very intense. Can China catch up?"})

AIMessage(content='中美在人工智能领域的竞争非常激烈。中国能赶得上吗？')

### ✍️ 尝试调用工具

In [287]:
# 定义一个简单的工具
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

@tool
def ask_neighber(query: str) -> str:
    """我在玩找人的游戏，我知道你找的人住在哪个房间"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

In [289]:
llm_zhipu = MiniZhipuAI().bind(tools=[convert_to_openai_tool(ask_neighber)])

In [290]:
llm_zhipu.invoke("告诉我马冬梅在哪个房间？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8513183423441263269', 'function': {'arguments': '{"query":"马冬梅"}', 'name': 'ask_neighber'}, 'type': 'function'}]})

In [291]:
ask_neighber.invoke({"query":"马冬梅"})

'楼上322'

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    openai_function 和 convert_to_openai_tool 的区别是什么？
</div>

**🌞 参考：**
- [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)


In [296]:
convert_to_openai_function(ask_neighber)

{'name': 'ask_neighber',
 'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [297]:
convert_to_openai_tool(ask_neighber)

{'type': 'function',
 'function': {'name': 'ask_neighber',
  'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string'}},
   'required': ['query']}}}

### ✍️ 尝试调用智能体

In [292]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
tools = [ask_neighber]

agent = create_openai_tools_agent(llm_zhipu, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [295]:
agent_executor.invoke({"input":"马冬梅住哪里"})



> Entering new AgentExecutor chain...

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322根据我的查询结果，马冬梅住在楼上322房间。希望这个信息对您有所帮助！

> Finished chain.


{'input': '马冬梅住哪里', 'output': '根据我的查询结果，马冬梅住在楼上322房间。希望这个信息对您有所帮助！'}

### 🌹 ChatZhipuAI

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    上面代码已经相对完整实现了一个 langchain 大模型；但缺少很多细节控制，可以尝试自己动手添加！
</div>

**🌞 参考：**
- [查看 langchain_zhpu 中的实现源码](https://github.com/arcstep/langchain_zhipuai/blob/e55af13eed673bc409ffdb143030e6cc0b2af27c/langchain_zhipu/chat.py#L304-L354)

In [299]:
from langchain_zhipu import ChatZhipuAI

llm_zhipuai = ChatZhipuAI()
for chunk in llm_zhipuai.stream("你是什么模型"):
    print(chunk.content, end="|")

我|使用的|模型|是|清华大学| K|EG| 实|验|室|和|智|谱|AI|共同|训练|的| GL|M| 模|型|，|一种|基于| Transformer| 的|通用|预|训练|语言|模型|。|Transformer| 模|型|是一种|基于|自|注意力|机制的|深度|神经网络|模型|，|经常|用于|处理|序列|数据|。

我|可能|用到|最大的|模型|是| GL|M|-|130|B|，|具有| |130|0| 亿|参数|，|支持|中|英|双语|。|我|具体|使用的|模型|规模|视|应用|场景|可能会有|所|变化|。||

In [300]:
from langchain_openai import ChatOpenAI

langchain_openai = ChatOpenAI()
for chunk in langchain_openai.stream("你是什么模型"):
    print(chunk.content, end="|")

|我|是|一个|基|于|人|工|智|能|技|术|的|对|话|模|型|，|可以|进行|自|然|语|言|交|互|并|提|供|相关|信息|和|帮|助|。|我|不|是|特|定|的|机|器|学|习|模|型|，|而|是|由|多|种|技|术|和|算|法|组|合|而|成|的|。||

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    事实上，看起来没有 stream 的 RunnableLambda 可以返回一个具有 stream 能力的 Runnable！你知道这是为什么？
</div>

**答案：**
- [查看 RunnableLambda 源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/runnables/base.py#L4056-L4071)

## 版本1

### 看看智谱官方的简单例子
### Langchain 相关库源码
### 写一个简单的 Langchain 版本
### 在LCEL中使用

## 版本2

### 试试流：暂时不支持
### 看看智谱官方对流的支持
### 看看流的默认实现源码
### 让我们也支持流

## 版本3

### 试试工具：暂时不支持
### 看看智谱官方对工具回调的支持
### 回顾工具回调：只是一种消息格式
### 看看 openai 实现源码
### 让我们也支持工具回调
### 在智能体中使用
### 完整代码请参考 langchain_zhipu

## 1、✍️ 代码准备：langchain中的LLM

In [ ]:
# LLM invoke
# OpenAI

# LLM stream
# OpenAI


### 2、✍️ 代码实践：如何实现一个FakeLLM

In [2]:
# 马冬梅楼下老大爷

[已经实现的FakeLLM](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/fake.py)

### 3、🌹 阅读源码：集成自己的大模型需要如何下手？

### 4、✍️ 代码实践：一步步集成智谱AI大模型

- 支持 invoke
- 支持 stream
- 支持 tools-calling

- 完整实现：[langchain_zhipu](https://github.com/arcstep/langchain_zhipu)

# ❤️（二）LLM输入输出：全面拆解 Runnable 结构

### 1、大模型调用过程中发生的数据流转换

- 从训过的模型说起：训练预料和预测文本风格（|<assistent>| xxxxxx |</assistent>|)
- 作为API提供：openAI风格
- 从提示语到langchain消息：BaseMessages
- 从langchain消息到大模型客户端：BaseChatModel
- 从大模型结果到langchain消息：BaseMessages
- 从langchain消息到输出解析器：OutpurParser

### 2、如果在这个过程中统一各模块标准，随时更换？

### 3、如果在这个过程中使用流、异步、批量、事件流 ... ？

### 4、这个过程中的缓存、重试、配置 ... ？

### 1、为什么说可以用 Runnable 组件在生产系统中搭积木？

- Runnable接口标准化的价值
    - 替换大模型：从GPT到国产云模型、到开源自训模型的多层次尝试和落地是必要的
    - 替换调用方式：满足业务验证、生产上线、业务扩容等多场景
    - 替换提示语模板：适应技术研究、业务定制、运营优化等多阶段
    - 替换解析器：适应模块调用、API调用等多协议对接
    - 替换回调集成：langsmith、langfuse云、langfuse本地化、自建运维系统
    - 替换向量数据库：适应不同场景、阶段的技术选型调整
    - 替换持久化：...
    - 替换智能体：...

### 2、🌹 阅读源码：Runnable 组件8个方法的实现逻辑

### 3、🌹 阅读源码：Runnable 组件配置自举能力

- 了解Runnable 组件的 8 个方法的默认实现
- 自定义Runnable（必要实现一般是invoke / stream / astream，或对应的内部函数）
- invoke：标准化调度
- batch: 标准化批量调度
- stream：标准化流式输出
- ainvoke / astream / abatch: 标准化异步调度
- astream_log / astream_events: 在链、智能体、langgraph等输出中按照names、tags、events提取流式日志
- config：统一管理配置
- schema：统一探查参数和配置

- 序列化

- 配置自举：在开放式应用中支持客户端自动识别自定义服务
- 容错：标准化重试策略
- 与langserve等api框架标准化对接
- 与langfuse等callbck框架标准化对接
- 与langchainjs等异构实现标准化对接


### 4、🌹 阅读源码：遗留的 Chain 是什么？

- langchain中提前写好的Chain资源

- 这些 Chain 局限性在究竟哪里？
    - 流程不灵活
    - 支持流式输出不彻底

### 5、✍️ 代码实践：若干种情况下的流输出

# ❤️ 第 2 部份：Agent

# ❤️（三）由LCEL实现智能体：全面拆解 LCEL 能力

### 1、LCEL 比 遗留 Chain 多哪些优势？

- LCEL 构建的替代 Chain
- 了解支撑LCEL的Runnable组件
    - Lambda
    - 迭代器
    - 字典和并行
    - 路由
    - 条件
    - 迭代执行
    - 绑定
    - 绘图

### 2、如何用 LCEL 定义智能体？

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor

### 3、🌹 阅读源码：了解 create_react_agent 的设计

- 一个简单的智能体需求：与AI玩一个捉迷藏游戏
- react智能体的prompt如何工作
- 中间步骤的一步步产生过程
- action如何解析

### 4、🌹 阅读源码：了解 AgentExecutor 的执行逻辑

### 4、✍️ 代码实践：如何用 AgentExcutor 再现《手撕AutoGPT》？

难点：
- 官方例子和内置智能体无法支持pydantic参数解析（智谱AI等推理能力较弱的模型可以使用）

# ❤️（四）由LangGraph实现智能体：全面拆解 LangGraph 能力

### 1、LangGraph 比 LCEL 多了什么？

### 2、如何使用 LangGraph 定义智能体？

### 3、🌹 阅读源码：了解 LangGraph 的执行逻辑

### 4、✍️ 代码实践：如何用 LangGraph 再现《手撕AutoGPT》？

难点：
- 官方例子和内置智能体无法支持流

# ❤️ 课程结束

## 1、课程总结

- 我们一起阅读了langchain的源代码结构和部份细节
    - BaseLanguageModel / BaseChatModel / 
    - Runnable
    - LambdaRunnable
    - Chain
    - AgentExcutor
    - langgraph.prebuild
- 我们学习了如何自己动手集成大模型到 langchain 中
- 我们拆解了langchain的基石组件：Runnable
- 我们拆解了langchain的核心逻辑能力：LCEL
- 我们拆解了langchain的最新逻辑能力：langgraph
- 我们动手做了一些代码实践

## 2、最后建议

- 技术选型时要对 Langchain 有绝对信心（几乎都不会是langchain的错）
- 内置链尽量使用LCEL链
- 内置智能体尽量使用 Langgraph
- 自定义智能体时使用 Langgraph
- 模块优先做成Runnable或LCEL链，其次再考虑Lambda
- 工具中包含大模型调用时优先做成Runnable或LCEL链，其次再考虑invoke


## 3、彩蛋

### 1、✍️ 代码实践：如何同时使用langchain的记忆和持久化

这是 langchain 文档中一个自相矛盾的地方，留给大家课后讨论。

### 2、✍️ 代码实践：如何将自己训的大模型集成到 langchain 中